In [ ]:
# Web scraping Canadian breweries from ratebeer.com/breweries/canada/0/39/

In [ ]:
from tqdm import tqdm
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import time

In [ ]:
# Set the selenium webdriver options
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('--disable-infobars')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--no-sandbox')
options.add_argument('--remote-debugging-port=9222')

In [ ]:
# Scraping brewery names, links to their ratebeer.com pages and types
col = ['brewery', 'link', 'type']
d = {cat : [] for cat in col}
url = 'https://www.ratebeer.com/breweries/canada/0/39/'

driver = webdriver.Chrome(options=options)
driver.get(url)
time.sleep(5)
soup = bs(driver.page_source, "html.parser")
driver.quit()

for i in tqdm(range(1535)): # 1535 active breweries at the time of scraping
    try: 
        d['brewery'].append(soup.select('.tab-pane')[0].select('.tablesorter')[0].select('tr')[i].select('a')[0].text.strip())
        d['link'].append(soup.select('.tab-pane')[0].select('.tablesorter')[0].select('tr')[i].select('a')[0]['href'])
        d['type'].append(soup.select('.tab-pane')[0].select('.tablesorter')[0].select('tr')[i].select('td')[1].text)
    except:
        pass

In [ ]:
# Getting brewery addresses and saving them into a text file
addresses = []
for i in tqdm(range(1535)):
    driver = webdriver.Chrome(options=options)
    driver.get(url + d['link'][i])
    time.sleep(5)
    soup = bs(driver.page_source, "html.parser")
    try:
        addresses.append(soup.select('.fd-r')[1].text)
    except:
        addresses.append('NaN')
    driver.quit()

file_path = 'addresses.txt'

# Open the file in write mode
with open(file_path, 'w') as file:
    # Write each item in the list to a new line
    for item in addresses:
        if item == 'NaN':
            file.write(item)
        else:
            file.write(item[2:]) # all address strings start with CA, which we don't need

In [ ]:
# Generating and saving a breweries dataframe
with open('addresses.txt', 'r') as file:
    # Read the lines of the file
    lines = file.readlines()

df = pd.DataFrame(data = d, columns=col)
df['address'] = lines
df.to_csv('breweries.csv', index=False)